<a href="https://colab.research.google.com/github/guanyu1127/Programming-Language/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
# PTT 電影版輿情分析機器人 (v6 - 最終成功版)

這個 Colab 筆記本整合了一個完整的自動化流程，包含：
1.  **爬取 PTT 電影版**：抓取指定頁數的文章標題。
2.  **寫入 Google Sheet**：將爬蟲結果儲存到 Google Sheet。
3.  **讀取與分析**：
    * **TF-IDF**：自動找出熱門關鍵字。
    * **自訂關鍵字**：統計您手動輸入的關鍵字出現次數。
4.  **回寫統計結果**：將兩種分析結果分別寫回 Google Sheet 的不同工作表。
5.  **AI 生成摘要**：使用經診斷確認可用的 Gemini API 模型 (gemini-pro-latest) 生成洞察摘要與結論。
6.  **Gradio 互動介面**：提供一個簡單的 Web UI 來一鍵執行以上所有流程。

請依照以下步驟執行。
"""

# @title 1. 安裝所有必要的套件
# @markdown ---
# @markdown 首先，執行此儲存格來安裝此專案所需的 Python 套件。
!pip install requests beautifulsoup4 gspread google-auth-oauthlib google-auth-httplib2 pandas jieba scikit-learn gradio google-generativeai -q

# @title 2. 匯入套件與設定
# @markdown ---
# @markdown 匯入所有會用到的 Python 函式庫。
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import google.generativeai as genai
import gradio as gr
import time

# @title 3. Google 服務授權
# @markdown ---
# @markdown 執行此儲存格以授權 Colab 存取您的 Google 帳戶，以便操作 Google Sheet。
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Google 服務授權成功！")
except Exception as e:
    print(f"🔴 授權失敗，請檢查 API 設定或重新執行。錯誤訊息：{e}")


# @title 4. 定義核心功能函式
# @markdown ---
# @markdown 這裡我們將爬蟲、Google Sheet 操作、文本分析和 Gemini API 串接等功能都封裝成獨立的函式。

# 4.1 爬蟲函式
def scrape_ptt_movie(pages_to_fetch: int):
    """
    爬取 PTT 電影版指定頁數的文章列表。
    """
    print(f"🚀 開始爬取 PTT 電影版，共 {pages_to_fetch} 頁...")
    articles_data = []
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 'Cookie': 'over18=1'}

    try:
        response_index = requests.get("https://www.ptt.cc/bbs/movie/index.html", headers=headers, timeout=10)
        soup_index = BeautifulSoup(response_index.text, 'html.parser')
        prev_button = soup_index.find('div', class_='btn-group-paging').find_all('a')[1]
        latest_index = int(re.search(r'index(\d+)\.html', prev_button['href']).group(1)) + 1

        for i in range(pages_to_fetch):
            current_index = latest_index - i
            url = f"https://www.ptt.cc/bbs/movie/index{current_index}.html"
            print(f"正在爬取第 {i+1}/{pages_to_fetch} 頁: {url}")
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code != 200:
                print(f"⚠️ 頁面 {url} 讀取失敗，狀態碼: {response.status_code}")
                continue
            soup = BeautifulSoup(response.text, 'html.parser')
            for article in soup.find_all('div', class_='r-ent'):
                title_tag = article.find('div', class_='title').find('a')
                if title_tag:
                    articles_data.append({
                        'title': title_tag.text.strip(),
                        'date': article.find('div', class_='date').text.strip(),
                        'author': article.find('div', class_='author').text.strip(),
                        'href': "https://www.ptt.cc" + title_tag['href']
                    })
            time.sleep(0.5)

        df = pd.DataFrame(articles_data)
        print(f"✅ 爬取完成，共抓取 {len(df)} 篇文章。")
        return df

    except Exception as e:
        print(f"🔴 爬蟲過程中發生錯誤: {e}")
        return pd.DataFrame()

# 4.2 Google Sheet 操作函式
def write_to_sheet(df: pd.DataFrame, sheet_name: str, worksheet_name: str):
    if df.empty:
        print(f"⚠️ DataFrame 是空的，不對 '{worksheet_name}' 執行寫入操作。")
        return
    print(f"📝 正在將資料寫入 Google Sheet '{sheet_name}' 的 '{worksheet_name}' 工作表...")
    try:
        sh = gc.open(sheet_name)
        try:
            worksheet = sh.worksheet(worksheet_name)
            worksheet.clear()
        except gspread.WorksheetNotFound:
            worksheet = sh.add_worksheet(title=worksheet_name, rows="1", cols="1")
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())
        print("✅ 資料寫入成功！")
    except Exception as e:
        print(f"🔴 寫入 Google Sheet ('{worksheet_name}') 時發生錯誤: {e}")

def read_from_sheet(sheet_name: str, worksheet_name: str):
    print(f"📥 正在從 Google Sheet '{sheet_name}' 的 '{worksheet_name}' 工作表讀取資料...")
    try:
        sh = gc.open(sheet_name)
        worksheet = sh.worksheet(worksheet_name)
        data = worksheet.get_all_records()
        df = pd.DataFrame(data)
        print(f"✅ 資料讀取成功！共 {len(df)} 筆。")
        return df
    except Exception as e:
        print(f"🔴 讀取 Google Sheet 時發生錯誤: {e}")
        return pd.DataFrame()

# 4.3 文本分析函式
def analyze_text_tfidf(df: pd.DataFrame, top_n: int):
    print(f"📊 正在進行 TF-IDF 分析，找出前 {top_n} 個關鍵字...")
    if df.empty or 'title' not in df.columns: return pd.DataFrame()
    stopwords = {'的', '了', '是', '在', '我', '你', '他', '她', '之', '一個', '和', '討論', '分享', '問題', 'Re', 'Fw'}
    corpus = df['title'].apply(lambda title: " ".join([word for word in jieba.lcut(re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s]', '', title)) if word.strip() and len(word) > 1 and word not in stopwords])).tolist()
    try:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(corpus)
        feature_names = vectorizer.get_feature_names_out()
        avg_tfidf_scores = tfidf_matrix.mean(axis=0).tolist()[0]
        sorted_keywords = sorted(zip(feature_names, avg_tfidf_scores), key=lambda x: x[1], reverse=True)
        top_keywords_df = pd.DataFrame(sorted_keywords[:top_n], columns=['keyword', 'tfidf_score']).round({'tfidf_score': 4})
        print("✅ TF-IDF 分析完成！")
        return top_keywords_df
    except ValueError as e:
        print(f"🔴 TF-IDF 分析失敗: {e}。可能是所有標題都被過濾掉了。")
        return pd.DataFrame()

def analyze_user_keywords(df: pd.DataFrame, user_keywords_str: str):
    print(f"🔎 正在統計自訂關鍵字...")
    if df.empty or 'title' not in df.columns or not user_keywords_str:
        print("⚠️ 無法進行自訂關鍵字統計 (資料為空或未輸入關鍵字)。")
        return pd.DataFrame()

    keywords = [kw.strip() for kw in re.split(r'[,，\s]+', user_keywords_str) if kw.strip()]
    if not keywords: return pd.DataFrame()

    keyword_counts = {kw: 0 for kw in keywords}
    for title in df['title']:
        for kw in keywords:
            if kw.lower() in title.lower():
                keyword_counts[kw] += 1

    counts_df = pd.DataFrame(list(keyword_counts.items()), columns=['user_keyword', 'count']).sort_values(by='count', ascending=False)
    print("✅ 自訂關鍵字統計完成！")
    return counts_df

# 4.4 Gemini API 相關函式
def test_gemini_api(api_key: str):
    if not api_key:
        return "🔴 請輸入您的 Gemini API 金鑰。"
    try:
        genai.configure(api_key=api_key)
        # --- ✨ 最終成功版修改 ✨ ---
        # 使用經診斷工具確認可用的 'gemini-pro-latest' 模型
        model = genai.GenerativeModel('gemini-pro-latest')
        response = model.generate_content("你好", generation_config=genai.types.GenerationConfig(
        candidate_count=1, max_output_tokens=5))
        return "✅ API 金鑰驗證成功！可以正常連線。"
    except Exception as e:
        return f"🔴 API 金鑰驗證失敗！\n\n錯誤訊息：{str(e)[:500]}...\n\n請參考介面上的「疑難排解」說明進行檢查。"

def generate_summary_with_gemini(api_key: str, keywords_df: pd.DataFrame):
    print("🤖 正在使用 Gemini API 生成摘要...")
    if keywords_df.empty: return "無法生成摘要，因為沒有提供關鍵字。", "無法生成結論。"
    try:
        genai.configure(api_key=api_key)
        # --- ✨ 最終成功版修改 ✨ ---
        # 使用經診斷工具確認可用的 'gemini-pro-latest' 模型
        model = genai.GenerativeModel('gemini-pro-latest')
        keywords_list = keywords_df['keyword'].tolist()
        prompt = f"你是一位專業的社群輿情分析師。請根據以下從 PTT 電影版文章標題中提取出的熱門關鍵字：[{', '.join(keywords_list)}]，執行以下兩項任務：\n\n任務一：產生五句條列式的市場洞察與趨勢分析。\n任務二：根據以上的洞察，撰寫一段約 120 字的總結。\n\n請用繁體中文回答，並直接提供結果。"
        response = model.generate_content(prompt)
        clean_text = response.text.replace('**', '').replace('*', '').strip()
        parts = re.split(r'總結：|結論：', clean_text)
        summary, conclusion = (parts[0].strip(), parts[1].strip()) if len(parts) > 1 else (clean_text, "結論無法從回應中分離。")
        print("✅ Gemini 摘要生成成功！")
        return summary, conclusion
    except Exception as e:
        print(f"🔴 Gemini API 呼叫失敗: {e}")
        return f"API 呼叫失敗: {e}", "無法生成結論。"

# @title 5. 整合主要工作流程
# @markdown ---
# @markdown 這個函式會將上述所有功能串連起來，形成一個完整的自動化流程。
def run_analysis_pipeline(pages: int, sheet_name: str, top_n: int, user_keywords_str: str, gemini_api_key: str):
    if not sheet_name or not gemini_api_key:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), "", "", "🔴 錯誤：請務必填寫 Google Sheet 檔案名稱與 Gemini API 金鑰。"

    scraped_df = scrape_ptt_movie(pages)
    if scraped_df.empty: return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), "", "", "🔴 爬蟲失敗，流程中止。"
    write_to_sheet(scraped_df, sheet_name, "爬蟲原始資料")

    read_df = read_from_sheet(sheet_name, "爬蟲原始資料")
    if read_df.empty: return scraped_df, pd.DataFrame(), pd.DataFrame(), "", "", "🔴 從 Google Sheet 讀取資料失敗，流程中止。"

    tfidf_keywords_df = analyze_text_tfidf(read_df, top_n)
    write_to_sheet(tfidf_keywords_df, sheet_name, "TF-IDF關鍵字統計")

    user_keywords_df = analyze_user_keywords(read_df, user_keywords_str)
    write_to_sheet(user_keywords_df, sheet_name, "自訂關鍵字追蹤")

    summary, conclusion = generate_summary_with_gemini(gemini_api_key, tfidf_keywords_df)

    status_message = "✅ 所有流程執行成功！"
    print(f"\n{'='*50}\n{status_message}\n{'='*50}\n")
    return scraped_df, tfidf_keywords_df, user_keywords_df, summary, conclusion, status_message


# @title 6. 建立 Gradio 使用者介面
# @markdown ---
# @markdown 執行此儲存格來啟動 Gradio 互動介面。
def create_gradio_interface():
    print("🚀 正在啟動 Gradio 介面...")
    with gr.Blocks(theme=gr.themes.Soft(), title="PTT 電影版輿情分析") as demo:
        gr.Markdown("# 📈 PTT 電影版輿情分析機器人 (v6)")

        with gr.Accordion("事前準備 & Gemini API 疑難排解", open=False):
            gr.Markdown(
                """
                ### **事前準備**
                1.  **建立全新 Google Cloud 專案**：為確保設定乾淨，建議建立一個全新的專案來產生 API 金鑰。
                2.  **啟用 API**：在新專案中，務必啟用 "Generative Language API"。
                3.  **建立全新 API 金鑰**：在新專案中，建立一組全新的 API 金鑰。
                4.  **分享權限**：將此 Colab 的服務帳號 Email (在第 3 步「Google 服務授權」的輸出中尋找) 加入為該 Google Sheet 的「**編輯者**」。
                """
            )

        gr.Markdown("### 🛠️ 步驟一：測試 Gemini API 金鑰 (可選，但建議初次使用時測試)")
        with gr.Row():
            gemini_api_key_test_input = gr.Textbox(label="在此貼上您的 Gemini API 金鑰進行測試", type="password")
            test_api_button = gr.Button("🧪 測試 API 金鑰")
        test_api_output = gr.Textbox(label="測試結果", interactive=False)
        test_api_button.click(fn=test_gemini_api, inputs=[gemini_api_key_test_input], outputs=[test_api_output])

        gr.Markdown("---")
        gr.Markdown("### ⚙️ 步驟二：輸入參數並執行完整流程")
        with gr.Row():
            with gr.Column(scale=1):
                pages_input = gr.Slider(minimum=1, maximum=20, value=5, step=1, label="要爬取的頁數")
                sheet_name_input = gr.Textbox(label="Google Sheet 檔案名稱", info="請確保名稱與您的檔案完全一致。")
                top_n_input = gr.Slider(minimum=5, maximum=20, value=10, step=1, label="TF-IDF 熱門關鍵字數量 (Top N)")
                user_keywords_input = gr.Textbox(label="請輸入您想追蹤的關鍵字", info="以逗號或空格分隔，例如：沙丘, 哥吉拉, 奧本海默")
                gemini_api_key_input = gr.Textbox(label="Gemini API 金鑰", type="password", info="執行完整流程需要再次輸入金鑰。")
                run_button = gr.Button("🚀 開始執行分析", variant="primary")

            with gr.Column(scale=2):
                status_output = gr.Textbox(label="執行狀態", interactive=False)
                with gr.Tab("AI 洞察與結論"):
                    summary_output = gr.Markdown(label="Gemini 洞察摘要")
                    conclusion_output = gr.Textbox(label="Gemini 結論", lines=4, interactive=False)
                with gr.Tab("TF-IDF 關鍵字統計"):
                    tfidf_output = gr.DataFrame(headers=["keyword", "tfidf_score"], label="自動分析熱詞")
                with gr.Tab("自訂關鍵字追蹤"):
                    user_keywords_output = gr.DataFrame(headers=["user_keyword", "count"], label="手動追蹤詞頻")
                with gr.Tab("爬蟲原始資料"):
                    scraped_data_output = gr.DataFrame(label="抓取的文章列表")

        run_button.click(
            fn=run_analysis_pipeline,
            inputs=[pages_input, sheet_name_input, top_n_input, user_keywords_input, gemini_api_key_input],
            outputs=[scraped_data_output, tfidf_output, user_keywords_output, summary_output, conclusion_output, status_output]
        )

    demo.launch(debug=True, share=True)

# 執行函式以啟動介面
create_gradio_interface()

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


✅ Google 服務授權成功！
🚀 正在啟動 Gradio 介面...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://55a78d2bd28484d59c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🚀 開始爬取 PTT 電影版，共 5 頁...
正在爬取第 1/5 頁: https://www.ptt.cc/bbs/movie/index10812.html
正在爬取第 2/5 頁: https://www.ptt.cc/bbs/movie/index10811.html
正在爬取第 3/5 頁: https://www.ptt.cc/bbs/movie/index10810.html
正在爬取第 4/5 頁: https://www.ptt.cc/bbs/movie/index10809.html
正在爬取第 5/5 頁: https://www.ptt.cc/bbs/movie/index10808.html
✅ 爬取完成，共抓取 96 篇文章。
📝 正在將資料寫入 Google Sheet 'HW4' 的 '爬蟲原始資料' 工作表...
✅ 資料寫入成功！
📥 正在從 Google Sheet 'HW4' 的 '爬蟲原始資料' 工作表讀取資料...


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...


✅ 資料讀取成功！共 96 筆。
📊 正在進行 TF-IDF 分析，找出前 10 個關鍵字...


Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.810 seconds.
DEBUG:jieba:Loading model cost 0.810 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


✅ TF-IDF 分析完成！
📝 正在將資料寫入 Google Sheet 'HW4' 的 'TF-IDF關鍵字統計' 工作表...
✅ 資料寫入成功！
🔎 正在統計自訂關鍵字...
✅ 自訂關鍵字統計完成！
📝 正在將資料寫入 Google Sheet 'HW4' 的 '自訂關鍵字追蹤' 工作表...
✅ 資料寫入成功！
🤖 正在使用 Gemini API 生成摘要...
✅ Gemini 摘要生成成功！

✅ 所有流程執行成功！

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://55a78d2bd28484d59c.gradio.live
